# Project: Data Cleaning and Manipulation with Pandas

In [1]:
import pandas as pd
import re
from funciones import injury

In [2]:
data = pd.read_csv('./input/GSAF5.csv', encoding = "ISO-8859-1")

In [ ]:
# Operaciones genéricas para conocer los datos

In [3]:
data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [4]:
data.tail()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
5987,ND.0005,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6,NaN,NaN
5988,ND.0004,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5,NaN,NaN
5989,ND.0003,1900-1905,0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4,NaN,NaN
5990,ND.0002,1883-1889,0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,...,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3,NaN,NaN
5991,ND.0001,1845-1853,0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,...,NaN,S.W. Baker,ND-0001-Ceylon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0001,ND.0001,2,NaN,NaN


In [5]:
data.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [6]:
data.shape

(5992, 24)

In [ ]:
# Qué columnas selecciono para mi análisis

In [7]:
remove_columns = ["Date", "Name", "Age", "Time", "Investigator or Source", "pdf",
                  "href formula", "href", "Case Number.1", "Case Number.2", 
                  "original order", "Unnamed: 22", "Unnamed: 23"]

clean_data = data.drop(columns=remove_columns)
clean_data.shape

(5992, 11)

In [9]:
# Eliminación de las filas que no me interesan

In [8]:
clean_data = clean_data.loc[clean_data["Year"] > 1989]
clean_data.shape

(2423, 11)

In [ ]:
# Comprobación de que los datos que tengo son a partir de 1990

In [9]:
clean_data.tail()

,Case Number,Year,Type,Country,Area,Location,Activity,Sex,Injury,Fatal (Y/N),Species
2419,1990.03.05,1990,Unprovoked,REUNION,Sainte-Marie,Baie de la Mare,Surfing,M,Survived,N,3 m [10'] bull shark
2420,1990.02.17,1990,Unprovoked,USA,Hawaii,"Mokapu, Kane'ohe Marine Air Corps Station, O'ahu",Scuba diving & spearfishing,M,Right arm severed FATAL,Y,Two sharks seen in vicinity: 2.4 m & 4.25 m [...
2421,1990.02.05,1990,Unprovoked,USA,Florida,"Monster Hole, Sebastian Inlet, Indian River Co...",Board sailing,M,Foot bitten,N,NaN
2422,1990.01.12,1990,Unprovoked,USA,California,"Montera Beach, San Mateo County",Surfing (sitting on his board),M,"No injury, board bitten",N,3 m to 4 m [10' to 13'] white shark
2423,1990.00.00,1990,Unprovoked,USA,Florida,"Pensacola, Escambia County",Surfing,M,NaN,UNKNOWN,NaN


In [ ]:
# Cambio de nombre de algunas columnas

In [10]:
clean_data = clean_data.rename(columns={"Species ":"Species", "Sex ":"Sex", "Fatal (Y/N)":"Fatal"})
clean_data.columns

Index(['Case Number', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Sex', 'Injury', 'Fatal', 'Species'],
      dtype='object')

In [15]:
# Comprobación de valores nulos en cada una de las columnas

In [11]:
null_cols = clean_data.isnull().sum()
null_cols[null_cols > 0]

Country       1
Area         97
Location    102
Activity    119
Sex         147
Injury        5
Fatal         2
Species     869
dtype: int64

In [ ]:
# Análsis información columna Type

In [12]:
set(clean_data["Type"])

{'Boat', 'Invalid', 'Provoked', 'Sea Disaster', 'Unprovoked'}

In [ ]:
# Comprobación del signifcado de cada una de las categorias

In [13]:
clean_data.loc[clean_data["Type"] == "Boat"].head()

,Case Number,Year,Type,Country,Area,Location,Activity,Sex,Injury,Fatal,Species
5,2016.09.15.R,2016,Boat,AUSTRALIA,Western Australia,Bunbury,Fishing,NaN,Shark rammed boat. No injury to occupant,N,NaN
22,2016.07.28,2016,Boat,AUSTRALIA,Western Australia,Near Albany,Kayaking,M,"No injury, shark nudged kayak repeatedly",N,White shark
29,2016.07.17,2016,Boat,USA,Alabama,8 miles off Mobile,Fishing in Alabama Deep Fishing Rodeo,NaN,"No injury, shark bit trolling motor",N,"Tiger shark, 10'"
35,2016.07.08,2016,Boat,USA,California,"Capitola, Santa Cruz County",Fishing for squid,M,"No injury. Hull bitten, tooth fragment recovered",N,White shark
37,2016.07.07.a,2016,Boat,USA,California,"Off Palos Verdes peninsula, Los Angeles County",Fishing for sharks,M,"No injury. Hull bitten, tooth fragment recovered",N,White shark


In [14]:
clean_data.loc[clean_data["Type"] == "Invalid"].head()

,Case Number,Year,Type,Country,Area,Location,Activity,Sex,Injury,Fatal,Species
50,2016.06.07,2016,Invalid,USA,South Carolina,"Folly Beach, Charleston County",Surfing,M,"No injury, board damaged",N,Said to involve an 8' shark but more likely da...
73,2016.04.08,2016,Invalid,CAPE VERDE,Boa Vista Island,NaN,NaN,M,"""Serious""",N,Shark involvement not confirmed
75,2016.04.07.a,2016,Invalid,USA,Florida,"Corners Beach, Jupiter, Palm Beach County",SUP,M,Fell off board when spinner shark leapt from t...,N,NaN
81,2016.03.13,2016,Invalid,USA,California,"Bolsa Chica State Park, Orange County",Surfing,NaN,Board reportedly bumped by shark. No injury,N,Shark involvement not confirmed
90,2016.02.10.R,2016,Invalid,CAYMAN ISLANDS,Grand Cayman,Stingray City Bar,Feeding stingrays?,M,Minor injury to wrist from Southern stingray,N,No shark involvement


In [15]:
clean_data.loc[clean_data["Type"] == "Provoked"].head()

,Case Number,Year,Type,Country,Area,Location,Activity,Sex,Injury,Fatal,Species
23,2016.07.27,2016,Provoked,USA,Florida,"Florida Keys, Monroe County",Lobstering,M,Laceration to left forearm PROVOKED INCIDENT,N,"Nurse shark, 4'"
28,2016.07.20,2016,Provoked,AUSTRALIA,Queensland,"20 k off The Spit, off the Gold Coast",Fishing,M,Laceration to left calf from hooked shark PROV...,N,"reef shark, 1m"
36,2016.07.07.b,2016,Provoked,USA,Massachusetts,"Off Gloucester, Essec County",Fishing,M,Fin of hooked shark injured fisherman's forear...,N,dogfish shark
39,2016.07.04,2016,Provoked,AUSTRALIA,Queensland,Palm Cove,Fishing,M,Right thigh injured by hooked pregnant female ...,N,Tawny nurse shark
47,2016.06.15.a,2016,Provoked,AUSTRALIA,Western Australia,Coral Bay,Spearfishing,M,No injury but shark punctured his wetsuit afte...,N,5' shark


In [16]:
clean_data.loc[clean_data["Type"] == "Sea Disaster"].head()

,Case Number,Year,Type,Country,Area,Location,Activity,Sex,Injury,Fatal,Species
243,2014.12.28.d,2014,Sea Disaster,GREECE,NaN,33 nautical miles off Othonoi Island,NaN,NaN,"Of 9 bodies recovered, one was bitten by a shark",N,Shark involvement prior to death still to be d...
365,2014.01.04,2014,Sea Disaster,JAPAN,Okinawa Prefecture,Off Miyako Island,Sea disaster,M,5 cm bite to left foot,N,NaN
668,2011.07.30,2011,Sea Disaster,PHILIPPINES,NaN,NaN,Sea disaster,M,The two Malabagos were bitten by sharks but su...,N,NaN
935,2009.02.08,2009,Sea Disaster,USA,Puerto Rico,Quebradillas,Air Disaster,M,It is probable that all 5 passengers died on i...,Y,NaN
966,2008.11.25,2008,Sea Disaster,PHILIPPINES,Batanes Provine,Luzon Strait,Sinking of the cargo ship Mark Jason,M,"Of the 20 crew, 4 were bitten by shark. None o...",N,NaN


In [17]:
clean_data.loc[clean_data["Type"] == "Unprovoked"].head()

,Case Number,Year,Type,Country,Area,Location,Activity,Sex,Injury,Fatal,Species
0,2016.09.18.c,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,Minor injury to thigh,N,NaN
1,2016.09.18.b,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,Lacerations to hands,N,NaN
2,2016.09.18.a,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,M,Lacerations to lower leg,N,NaN
3,2016.09.17,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,M,Struck by fin on chest & leg,N,NaN
4,2016.09.15,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,M,No injury: Knocked off board by shark,N,2 m shark


In [ ]:
'''
Por "Boat" se clasifican aquellos accidentes o sucesos en los que un tiburón intentó y/o mordió una embarcación
o la embistió.

Por "Invalid" se clasifican aquellos accidentes o sucesos dudosos.

Por "Provoked" se clasifican aquellos en los que alguna persona (bañista, submarinista...)
provocó de alguna forma al animal.

Por "Unprovoked" se clasifican aquellos sucesos registrados en habitats naturales
de los tiburones pero sin que medie provocación por parte de una persona

Y por "Sea disaster" se clasifican aquellos sucesos derivados de uno previo, por ejemplo,
el hundimiento de un barco, algún accidente aéreo...


'''

In [19]:
# Supresión de las filas con valores "Invalid"

In [18]:
clean_data = clean_data.loc[clean_data["Type"] != "Invalid"]
clean_data.shape

(2243, 11)

In [ ]:
# Limpieza de columnas

In [ ]:
# Limpieza de la columna Sex. Podría ser interesante hacer una comparativa entre hombres y mujeres

In [19]:
set(clean_data["Sex"])

{'F', 'M', 'M ', 'lli', nan}

In [20]:
clean_data ["Sex"] = clean_data["Sex"].replace("M ", "M")

In [21]:
# El valor Lli puede ser un error al pasar los datos
clean_data.loc[clean_data["Sex"] == "lli"] 

,Case Number,Year,Type,Country,Area,Location,Activity,Sex,Injury,Fatal,Species
1400,2004.11.11.b,2004,Unprovoked,USA,California,"Bunkers, Humboldt Bay, Eureka, Humboldt County",Surfing,lli,"Lacerations to hand, knee & thigh",N,5.5 m [18'] white shark


In [22]:
clean_data["Sex"] = clean_data["Sex"].fillna(0)
clean_data["Sex"] = clean_data["Sex"].replace("lli", 0)
set(clean_data["Sex"])

{0, 'F', 'M'}

In [ ]:
# Limpieza de la columna "Fatal"

In [23]:
set(clean_data["Fatal"])

{'N', 'UNKNOWN', 'Y', nan}

In [24]:
clean_data["Fatal"] = clean_data["Fatal"].replace('UNKNOWN', 0)
clean_data["Fatal"] = clean_data["Fatal"].fillna(0)
set(clean_data["Fatal"])

{0, 'N', 'Y'}

In [ ]:
# Limpieza de la columna "Injury"

In [25]:
len(set(clean_data["Injury"]))

1407

In [26]:
clean_data["Injury"]

0                       Minor injury to thigh
1                        Lacerations to hands
2                    Lacerations to lower leg
3                Struck by fin on chest & leg
4       No injury: Knocked off board by shark
                        ...                  
2419                                 Survived
2420                 Right arm severed  FATAL
2421                              Foot bitten
2422                  No injury, board bitten
2423                                      NaN
Name: Injury, Length: 2243, dtype: object

In [27]:
print(len(clean_data.loc[clean_data["Injury"] == "FATAL"]))
print(len(clean_data.loc[clean_data["Fatal"] == "Y"]))

169
250


In [ ]:
# Por la disparidad de valores. Actualizo valores de Injury a "Fatal" cuando["Fatal"] == "Y" 

In [28]:
clean_data.loc[clean_data["Fatal"] == "Y", "Injury"] = "Fatal"

In [29]:
len(set(clean_data["Injury"]))

1335

In [30]:
set(clean_data["Injury"])

{nan,
 ' 6 lacerations to left hand',
 'Laceration to shin of left leg',
 'Upper left thigh avulsed  ',
 'Lacerations to upper lip    PROVOKED INCIDENT',
 'Right foot & heel lacerated',
 '9-inch gash in left foot ',
 'Severe lacerations to shoulder & forearm',
 'Lacerations to lower right calf',
 '3-inch laceration to right foot',
 'Leg injured',
 'Left elbow and bicep bitten',
 'Lower left calf & ankle  lacerated',
 'No injury, but fin lost from surfboard',
 'Calf lacerated',
 'No injury but shark bit trolling motor & rammed boat',
 'Laceration to leg',
 'No injury, shark struck board & spun it around',
 'minor injury to foot',
 'Lacerations & puncture wound  to arm',
 'Non-swimmer pulled off pier& into the water by a hooked shark PROVOKED INCIDENT',
 'No injury to occupant; shark bumped the kayak repeatedly for 15 minutes',
 'Puncture wounds to hand',
 'Minor cuts to right buttock & thigh',
 'Right foot & toes lacerated',
 '2 small cuts on right ear & head when he collided with the c

In [75]:
# Uso de una función para agrupar con regular expressions las posibles categorías. 

In [31]:
clean_data["Injury"] = clean_data["Injury"].apply(injury)
print(len(set(clean_data["Injury"])))

8


In [32]:
set(clean_data["Injury"])

{'Bitten',
 'Fatal',
 'Injured',
 'No injury',
 'No personal injury',
 'No shark',
 'Severed',
 'Unknown'}

In [ ]:
'''
Al analizar los valores de la columna "Injury", algunos de los accidentes se desligan
del ataque de un tiburón (por ejemplo, en uno de ellos se asegura que fue un delfín).
Por esta razón elimino las filas correspondientes al valor "No shark" de la columna "Injury"

'''

In [35]:
clean_data = clean_data.loc[clean_data["Injury"] != "No shark"]
clean_data.shape

(2241, 11)

In [ ]:
# Creación del csv

In [36]:
clean_data.to_csv('./output/sharks.csv', index=None, header=True)

In [ ]:
# Análisis y conclusiones de los datos

In [37]:
# Cantidad de accidentes por categoría. La amplia mayoría (1954) no son provocados
clean_data.groupby(["Type"]).size()

Type
Boat              98
Provoked         170
Sea Disaster      19
Unprovoked      1954
dtype: int64

In [95]:
# Cantidad de muertes por tipo de accidente. En todos los supuestos a excepción de aquellos
# ligados a otros sucesos la supervivencia supera holgadamente a las muertes
pd.crosstab(clean_data["Type"], clean_data["Fatal"])

Fatal,0,N,Y
Type,,,
Boat,0,96,2
Provoked,0,166,4
Sea Disaster,0,8,11
Unprovoked,14,1707,233


In [45]:
# Distribución del tipo de heridas en función del tipo de accidente. 
# El grupo más numeroso es la categoría "Injured", un grupo en el que he ido agregando
# todo tipo de heridas, desde arañazos, cortes, magulladuras, pinchazos, roturas...
# En el periodo de tiempo analizado, desde 1990 hasta 2016, en 114 de los ataques registrados
# la víctima ha tenido que sufrir alguna amputación (categoría "Severed")
# En 47 de los casos no se supo especificar el tipo de heridas
pd.crosstab(clean_data["Type"], clean_data["Injury"])

Injury,Bitten,Fatal,Injured,No injury,No personal injury,Severed,Unknown
Type,,,,,,,
Boat,2,2,8,85,1,0,0
Provoked,61,4,80,14,0,6,5
Sea Disaster,5,11,2,1,0,0,0
Unprovoked,490,233,862,213,1,108,47


In [43]:
clean_data.groupby(["Injury"]).size()

Injury
Bitten                558
Fatal                 250
Injured               952
No injury             313
No personal injury      2
Severed               114
Unknown                52
dtype: int64

In [99]:
# La evolución del acumulado de accidentes por año dibuja una línea ascendente
# Aunque no creo que esa subida se pueda achacar simplemente a una subida de los accidentes o ataques
# En la recolección de los datos han influido diversas mejoras, así como una mayor acceso
# a la información en general. Muchas de las fuentes utilizadas son medios de comunicación
# y gracias a Internet, por ejemplo, se puede acceder a diversos medios de multitud de países
# Otra conclusión que se podría extraer sería un aumento de la imprudencia, si bien la recolección
# de estos datos también está ligada a una mejora en la obtención de la información en general

pd.crosstab(clean_data["Type"], clean_data["Year"])


Year,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
Type,,,,,,,,,,,,,,,,,,,,,
Boat,1,1,3,5,1,0,1,3,0,3,...,3,6,6,1,4,3,1,10,9,7
Provoked,1,5,1,1,3,3,3,0,5,3,...,10,10,18,11,9,8,9,13,6,8
Sea Disaster,0,0,0,2,1,0,1,0,0,0,...,2,3,1,0,1,0,0,2,0,0
Unprovoked,34,31,46,47,49,65,54,51,58,55,...,86,92,84,85,103,97,105,91,111,81
